In [1]:
import h5py, os, pystan
import numpy as np
from os.path import dirname
import pandas as pd
from pandas import DataFrame, read_csv, concat
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

In [79]:
df = pd.read_csv('two_step_data.txt')

In [69]:
print(df)

                               subject  trial  drift_1  drift_2  drift_3  \
0       3018Q3ZVOIQGSXZ9L4SKHNWWZ1CRAO      1    0.743    0.273    0.717   
1       3018Q3ZVOIQGSXZ9L4SKHNWWZ1CRAO      2    0.714    0.290    0.712   
2       3018Q3ZVOIQGSXZ9L4SKHNWWZ1CRAO      3    0.709    0.346    0.714   
3       3018Q3ZVOIQGSXZ9L4SKHNWWZ1CRAO      4    0.732    0.339    0.704   
4       3018Q3ZVOIQGSXZ9L4SKHNWWZ1CRAO      5    0.717    0.408    0.722   
...                                ...    ...      ...      ...      ...   
341994  3ZY8KE4ISJ308KN7JKUG8K04OS9VQW    196    0.384    0.712    0.479   
341995  3ZY8KE4ISJ308KN7JKUG8K04OS9VQW    197    0.373    0.734    0.460   
341996  3ZY8KE4ISJ308KN7JKUG8K04OS9VQW    198    0.440    0.748    0.461   
341997  3ZY8KE4ISJ308KN7JKUG8K04OS9VQW    199    0.471    0.737    0.468   
341998  3ZY8KE4ISJ308KN7JKUG8K04OS9VQW    200    0.487    0.736    0.447   

        drift_4  stage_1_choice  stage_1_rt  transition  stage_2_choice  \
0         0.

In [70]:
from tqdm import tqdm

In [71]:
import seaborn as sns
sns.set('notebook', style='white', font='sans-serif', font_scale=1.33)

In [80]:
df = df.dropna()

In [81]:
df['stage_1_choice'] = df['stage_1_choice']-1

In [82]:
df['stage_1_choice']

1         0.0
2         0.0
3         0.0
4         0.0
5         0.0
         ... 
341994    0.0
341995    0.0
341996    0.0
341997    0.0
341998    1.0
Name: stage_1_choice, Length: 336450, dtype: float64

In [83]:
df.sort_index(inplace=True)

In [84]:
trial = df['trial']
reward = df['outcome']
Y1 = df['stage_1_choice']
transition = df['transition']

In [85]:
np.unique(Y1)

array([0., 1.])

In [86]:
np.unique(transition)

array([0., 1.])

In [87]:
t = np.where(Y1 == transition, 1, 0)

In [88]:
len(t)

336450

In [98]:
Y1_1shift = Y1[1:]

In [99]:
Y1_backshift = Y1[:-1]

In [100]:
len(Y1_1shift)

336449

In [101]:
len(Y1_backshift)

336449

In [102]:
stay = np.where(Y1_1shift == Y1_backshift, 1, 0)

ValueError: Can only compare identically-labeled Series objects

In [13]:
# t = whether it's common or rare
# Y1 = stage 1 choice

In [26]:
np.random.seed(47404)

data = []
for _ in tqdm(range(1500)):
    
    ## Define variables
    prev_R = np.roll(reward, 1)[1:]
    pret_T = np.roll(t, 1)[1:]
    stay = Y1[:-1] == Y1[1:]

    ## Dataframe assembly
    gf = DataFrame(np.column_stack([prev_R, prev_T, stay]), columns=['prev_R', 'prev_T', 'Stay'])
    data.append(gf)
    
## Concatenate DataFrames.
data = concat(data)
data.prev_R = data.prev_R.replace({1:'rewarded',0:'unrewarded'})
data.prev_T = data.prev_T.replace({1:'common',0:'uncommon'})

ax = sns.barplot('prev_R', 'Stay', 'prev_T', data=data, order=['rewarded','unrewarded'], 
                 hue_order=['common','uncommon'])

ax.set(xticklabels=['Common','Rare'], xlabel='Transition', ylabel='Stay')
ax.legend(loc=10, bbox_to_anchor=(1.25,0.5), frameon=False, title='Reward')

sns.despine()
plt.tight_layout()

  0%|          | 0/1500 [00:00<?, ?it/s]


ValueError: Can only compare identically-labeled Series objects